In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/Shareddrives/CS224s/data/AI_open_mic_dataset')

Mounted at /content/drive


In [ ]:
# loop over funny_texts
# loop over unfunny_texts
text_file_path = 'funny_text/AJ_TP_text_01.txt'
with open(text_file_path, 'r') as file:
    # Read the entire content of the file
    content = file.read()
print(content)


Thank you. Thank you. Thank you, San Francisco. Thank you so much. So good to be here. People were surprised when I told ’em I was gonna tape my special in San Francisco. Said, “Why would you do that? That’s the most politically correct city in the world.” Not when I’m on stage, it’s not.I love this place. One of my favorite cities to perform in. It’s a good place to hang out. Got to walk around a little bit today. Saw a baby. Saw a baby locked inside the back of a hot car. So it’s been a great day. Love that. Don’t get me wrong. I’m not a monster. I tried to help the baby. Tried to throw a rock through the window. Window was down. Ruined that baby’s whole weekend. It was worth it.


In [ ]:
def save_transcriptions(folder_path, transcription_list):
  for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)

        # Open and read the content of the file
        with open(file_path, 'r') as file:
            content = file.read()
            transcription_list.append(content)

In [ ]:
funny_folder_path = 'funny_text'
funny_transcriptions = []
save_transcriptions(funny_folder_path, funny_transcriptions)

In [ ]:
funny_transcriptions[0]

'Do you guys have any kids? How many? I have one, and he has two. Okay. You guys ever done that thing where you go online and see how many sex offenders live around you? Isn’t it crazy? Isn’t it way more than you’d think? You think it might be like one or two. No, it’s like chicken pox. I went online, 15 child molesters. Fifteen child molesters within five miles of my apartment. So, why do we always have to meet at my place? And if you’re wondering, “Anthony, this is great, but are all of these jokes gonna be about hurting children?” I don’t know, probably. Seems like something I would do.I of course live in Los Angeles. I’ve got a nice place. Obviously.'

In [ ]:
unfunny_folder_path = 'unfunny_text'
unfunny_transcriptions = []
save_transcriptions(unfunny_folder_path, unfunny_transcriptions)

In [ ]:
unfunny_transcriptions[0]

'So look, just six months ago it seems a lifetime ago, but it really was just six months ago -- climate seemed to be on the lips of every thinking person on the planet. Recent events seem to have swept it all away from our attention. How worried are you about that?\nWell, first of all Chris, thank you so much for inviting me to have this conversation. People are reacting differently to the climate crisis in the midst of these other great challenges that have taken over our awareness, appropriately. One reason is something that you mentioned. People get the fact that when scientists are warning us in ever more dire terms and setting their hair on fire, so to speak, it\'s best to listen to what they\'re saying, and I think that lesson has begun to sink in in a new way. Another similarity, by the way, is that the climate crisis, like the COVID-19 pandemic, has revealed in a new way the shocking injustices and inequalities and disparities that affect communities of color and low-income com

In [ ]:
# Create labels for each list
import pandas as pd
funny_labels = [1] * len(funny_transcriptions)  # List with label 1
unfunny_labels = [0] * len(unfunny_transcriptions)  # List with label 0

# Combine the lists and the labels
combined_strings = funny_transcriptions + unfunny_transcriptions
combined_labels = funny_labels + unfunny_labels

# Create the DataFrame
df_plain_text = pd.DataFrame({
    'text': combined_strings,
    'label': combined_labels
})


In [ ]:
print(f"funny counts are {len(funny_labels)} and unfunny counts are {len(unfunny_labels)}")

funny counts are 491 and unfunny counts are 96


In [ ]:
df_plain_text

,text,label
0,Do you guys have any kids? How many? I have on...,1
1,"Now, this next joke, this next joke is a test ...",1
2,"Now, I gotta get to know you guys a little bit...",1
3,I love San Francisco. One of my favorite thing...,1
4,"Thank you. Thank you. Thank you, San Francisco...",1
...,...,...
582,"I think, again, that fed into the anger. It fe...",0
583,Do you think enough of his supporters have a g...,0
584,How do we rebuild communities in this modern a...,0
585,I think you're being too optimistic.\nYou do? ...,0


In [ ]:
# create sentence/paragraph embeddings of each data
!pip install transformers
from transformers import DistilBertTokenizer, DistilBertModel
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Function to generate embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the mean pooling of the last hidden state
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings[0].numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
df_plain_text['embedding'] = df_plain_text['text'].apply(get_embedding)

In [ ]:
# save the embedding file
file_path = '/content/drive/Shareddrives/CS224s/df_text_embeddings_data.csv'

# Save the DataFrame to the specified file path
df_plain_text.to_csv(file_path, index=False)


### Train Classifier with 491 Positive and 96 Negative Examples

In [ ]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [ ]:
X = torch.tensor(df_plain_text['embedding'].tolist(), dtype=torch.float32)
y = torch.tensor(df_plain_text['label'].tolist(), dtype=torch.float32)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple binary classifier
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

<ipython-input-24-b38fa2482e0c>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  X = torch.tensor(df_plain_text['embedding'].tolist(), dtype=torch.float32)


In [ ]:
# Initialize the model, loss function, and optimizer
input_dim = X_train.shape[1]
model = BinaryClassifier(input_dim)
criterion = nn.BCELoss(reduction='none')  # We'll apply weights manually
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Calculate weights for the imbalanced classes
class_counts = torch.bincount(y_train.long())
class_weights = 1.0 / class_counts.float()
weights = class_weights[y_train.long()]

In [ ]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    outputs = outputs.squeeze()

    # Calculate weighted loss
    loss = criterion(outputs, y_train)
    loss = loss * weights  # Apply weights
    loss = loss.mean()  # Average the loss

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_train).float().mean()

    # Print loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Training Accuracy: {accuracy.item():.4f}')


Epoch [1/20], Loss: 0.0029, Training Accuracy: 0.1578
Epoch [2/20], Loss: 0.0027, Training Accuracy: 0.8380
Epoch [3/20], Loss: 0.0026, Training Accuracy: 0.9808
Epoch [4/20], Loss: 0.0024, Training Accuracy: 0.9957
Epoch [5/20], Loss: 0.0023, Training Accuracy: 0.9957
Epoch [6/20], Loss: 0.0021, Training Accuracy: 0.9957
Epoch [7/20], Loss: 0.0020, Training Accuracy: 0.9957
Epoch [8/20], Loss: 0.0018, Training Accuracy: 0.9957
Epoch [9/20], Loss: 0.0017, Training Accuracy: 0.9957
Epoch [10/20], Loss: 0.0015, Training Accuracy: 0.9979
Epoch [11/20], Loss: 0.0014, Training Accuracy: 0.9979
Epoch [12/20], Loss: 0.0013, Training Accuracy: 0.9979
Epoch [13/20], Loss: 0.0012, Training Accuracy: 0.9979
Epoch [14/20], Loss: 0.0011, Training Accuracy: 0.9979
Epoch [15/20], Loss: 0.0010, Training Accuracy: 0.9979
Epoch [16/20], Loss: 0.0009, Training Accuracy: 0.9979
Epoch [17/20], Loss: 0.0008, Training Accuracy: 0.9979
Epoch [18/20], Loss: 0.0008, Training Accuracy: 0.9979
Epoch [19/20], Loss

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_val).squeeze()
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_val).float().mean()
    print(f'Validation Accuracy: {accuracy.item():.4f}')

Validation Accuracy: 0.9915


### Classification with 100 positive 96 negative

In [ ]:
# Assuming df is the DataFrame with 'embedding' and 'label' columns
# Convert embeddings and labels to tensors
X = torch.tensor(df_plain_text['embedding'].tolist(), dtype=torch.float32)
y = torch.tensor(df_plain_text['label'].tolist(), dtype=torch.float32)

# Separate the indices for each class
class_0_indices = (y == 0).nonzero(as_tuple=True)[0]
class_1_indices = (y == 1).nonzero(as_tuple=True)[0]

# Randomly sample 100 indices from each class
sampled_class_0_indices = class_0_indices[torch.randperm(len(class_0_indices))[:100]]
sampled_class_1_indices = class_1_indices[torch.randperm(len(class_1_indices))[:100]]

# Combine the sampled indices
sampled_indices = torch.cat([sampled_class_0_indices, sampled_class_1_indices])

# Create the sampled dataset
X_sampled = X[sampled_indices]
y_sampled = y[sampled_indices]

# Split the sampled data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42)

# Define a simple binary classifier
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Initialize the model, loss function, and optimizer
input_dim = X_train.shape[1]
model = BinaryClassifier(input_dim)
criterion = nn.BCELoss()  # No need for manual weighting now
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train)
    outputs = outputs.squeeze()

    # Calculate loss
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Calculate training accuracy
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_train).float().mean()

    # Print loss and accuracy for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Training Accuracy: {accuracy.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_val).squeeze()
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_val).float().mean()
    print(f'Validation Accuracy: {accuracy.item():.4f}')

Epoch [1/20], Loss: 0.6895, Training Accuracy: 0.4936
Epoch [2/20], Loss: 0.6476, Training Accuracy: 0.9936
Epoch [3/20], Loss: 0.6127, Training Accuracy: 1.0000
Epoch [4/20], Loss: 0.5771, Training Accuracy: 1.0000
Epoch [5/20], Loss: 0.5421, Training Accuracy: 1.0000
Epoch [6/20], Loss: 0.5075, Training Accuracy: 1.0000
Epoch [7/20], Loss: 0.4737, Training Accuracy: 1.0000
Epoch [8/20], Loss: 0.4411, Training Accuracy: 1.0000
Epoch [9/20], Loss: 0.4096, Training Accuracy: 1.0000
Epoch [10/20], Loss: 0.3794, Training Accuracy: 1.0000
Epoch [11/20], Loss: 0.3505, Training Accuracy: 1.0000
Epoch [12/20], Loss: 0.3231, Training Accuracy: 1.0000
Epoch [13/20], Loss: 0.2972, Training Accuracy: 1.0000
Epoch [14/20], Loss: 0.2730, Training Accuracy: 1.0000
Epoch [15/20], Loss: 0.2503, Training Accuracy: 1.0000
Epoch [16/20], Loss: 0.2293, Training Accuracy: 1.0000
Epoch [17/20], Loss: 0.2098, Training Accuracy: 1.0000
Epoch [18/20], Loss: 0.1919, Training Accuracy: 1.0000
Epoch [19/20], Loss